# 07 – Deployment with Streamlit

This notebook demonstrates how to deploy the best model using a simple Streamlit app.  The app allows a user to enter input features and receive a predicted CTR.


In [1]:
import streamlit as st
import joblib
import pandas as pd
import os

# ===============================
# Paths
# ===============================
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
models_dir = os.path.join(project_root, 'models')
processed_dir = os.path.join(project_root, 'data', 'processed')

MODEL_NAME = "Stacking_model.pkl"   # change if another model is best
model_path = os.path.join(models_dir, MODEL_NAME)

# ===============================
# Load model (Pipeline)
# ===============================
@st.cache_resource
def load_model():
    return joblib.load(model_path)

model = load_model()

st.set_page_config(page_title="CTR Prediction", layout="centered")

st.title("📊 CTR Prediction App")
st.write(
    """
    This application predicts the probability that a user will click on an advertisement
    based on user, ad, and behavioral features.
    """
)

# ===============================
# Input section
# ===============================
st.header("Input Features")

# ---- User features ----
st.subheader("User Information")
user = st.number_input("User ID", min_value=0, step=1)
age_level = st.selectbox("Age Level", [0, 1, 2, 3, 4, 5, 6])
gender = st.selectbox("Gender", ["1", "2"])  # 1=Male, 2=Female
shopping_level = st.selectbox("Shopping Level", [1, 2, 3])
pvalue_level = st.selectbox("Consumption Level", [1, 2, 3])

# ---- Ad features ----
st.subheader("Ad Information")
adgroup_id = st.number_input("Ad Group ID", min_value=0, step=1)
campaign_id = st.number_input("Campaign ID", min_value=0, step=1)
cate_id = st.number_input("Category ID", min_value=0, step=1)
brand = st.number_input("Brand ID", min_value=0, step=1)
price = st.number_input("Price", min_value=0.0, step=1.0)

# ---- Behavior features (aggregated) ----
st.subheader("User Behavior (Aggregated)")
buy = st.number_input("Buy count", min_value=0, step=1)
cart = st.number_input("Cart count", min_value=0, step=1)
fav = st.number_input("Favorite count", min_value=0, step=1)
pv = st.number_input("Page views", min_value=0, step=1)

# ===============================
# Prediction
# ===============================
if st.button("Predict CTR"):
    # IMPORTANT:
    # Column names MUST match training data BEFORE preprocessing
    input_df = pd.DataFrame({
        "user": [user],
        "adgroup_id": [adgroup_id],
        "campaign_id": [campaign_id],
        "cate_id": [cate_id],
        "brand": [brand],
        "price": [price],
        "age_level": [age_level],
        "final_gender_code": [gender],
        "shopping_level": [shopping_level],
        "pvalue_level": [pvalue_level],
        "buy": [buy],
        "cart": [cart],
        "fav": [fav],
        "pv": [pv],
    })

    try:
        proba = model.predict_proba(input_df)[0, 1]

        st.success(f"✅ Predicted Click Probability: **{proba:.4f}**")

        if proba >= 0.3:
            st.info("📈 High click likelihood (above threshold)")
        else:
            st.warning("📉 Low click likelihood")

    except Exception as e:
        st.error("Prediction failed. Check feature compatibility.")
        st.exception(e)


2025-12-14 23:17:37.903 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 23:17:38.339 
  command:

    streamlit run d:\projects\Ai\project_fusion_ecu\.venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-12-14 23:17:38.340 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 23:17:38.341 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 23:17:38.342 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 23:17:38.856 Thread 'Thread-3': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 23:17:38.858 Thread 'Thread-3': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 23:17:38.858 Thread 'Thre